# Thust Wedge Tutorial
This tutorial is mostly based off Ruh et al. (2013) and other similar stuides, which examine the development thrust sheet and or accretionary wegdes with visco-plastic rheologies .In this 2D model, the boundary conditions, initial conditions,geometry material properties are based on Ruh et al. and other such similar studies.(see Ruh et al. 2013 and references therein: J.B. Ruh, T. Gerya nd J.-P. Burg (2013), G3, v.14(4), p. 1131-1155)

In [ ]:
import os
import UWGeodynamics as GEO
import numpy as np
import glucifer

### Model Scalling

In [ ]:
u = GEO.UnitRegistry

velocity = 1 * u.centimeter / u.year
model_length = 100. * u.kilometer
bodyforce = 2700. * u.kilogram / u.metre**3 * 9.81 * u.meter / u.second**2

KL = model_length
Kt = KL / velocity
KM = bodyforce * KL**2 * Kt**2

GEO.scaling["[length]"] = KL
GEO.scaling["[time]"] = Kt
GEO.scaling["[mass]"] = KM

In [ ]:
Model = GEO.Model(
    elementRes=(256 , 64),# for better shearbands use 512x128
    minCoord=(0. * u.kilometer, -7 * u.kilometer),
    maxCoord=(128. * u.kilometer, 9. * u.kilometer),
    gravity=(0.0, -9.81 * u.meter / u.second**2))

Model.outputDir = "outputs_tutorial9"


### Material Setup

In [ ]:
stickyAirLayer = Model.add_material(
    name="Air", shape=GEO.shapes.Layer(top=Model.top, bottom=0. * u.kilometer))


plTop = stickyAirLayer.bottom
plBot = -6 * u.kilometer
plasticLayerThickness = plTop - plBot

plasticLayerheight = .5 * u.kilometer
plasticLayerNos = plasticLayerThickness / plasticLayerheight
plasticLayers = [Model.add_material(name='Plastic' + str(i),
       shape=GEO.shapes.Layer(top=plTop - i * plasticLayerheight,
       bottom=plTop - (i * plasticLayerheight + plasticLayerheight)))
       for i in range(int(plasticLayerNos.magnitude))]


frictionalBasalLayer = Model.add_material(
    name="Basal_Frictional_Layer",
    shape=GEO.shapes.Layer(
        top=plasticLayers[-1].bottom, bottom=plasticLayers[-1].bottom-.5 * u.kilometer))

rigidBase = Model.add_material(
    name="Rigid_Base",
    shape=GEO.shapes.Layer(top=frictionalBasalLayer.bottom, bottom=Model.bottom))
sediment = Model.add_material(name="Sediment")

In [ ]:
figSize =(1200,250)
fnSize = 1.8
figQuality=3
store=None
# store=Model.outputDir+'/FigStore.gldb'
layeredColours=int(plasticLayerNos/2)*'#425e6a salmon '+'tan #3A3A3A #00a8a8'
bBox=((GEO.nd(Model.minCoord[0]), GEO.nd(Model.minCoord[1])),
                 (GEO.nd(Model.maxCoord[0]), GEO.nd(Model.maxCoord[1])))

figMaterial = glucifer.Figure(store, figsize=figSize, quality=figQuality,boundingBox=bBox,
                              name="Materials", title="Material Map")
figMaterial.Points(Model.swarm, Model.materialField,fn_size=fnSize,
                   colours=layeredColours,discrete=True,
                   valueRange=[plasticLayers[0].index, sediment.index],
                   fn_mask=Model.materialField > stickyAirLayer.index)
figMaterial.objects[0].colourBar["binlabels"] = True
figMaterial.show()

In [ ]:
stickyAirLayer.density = 1. * u.kilogram / u.metre**3
for plasticLayer in plasticLayers:
    plasticLayer.density = 2700. * u.kilogram / u.metre**3
frictionalBasalLayer.density = 2700. * u.kilogram / u.metre**3
rigidBase.density = 2700. * u.kilogram / u.metre**3
sediment.density = 2700. * u.kilogram / u.metre**3

### Model Rheology, a basic Visco-Plastic Rheology of Drucker Prager Type

In [ ]:
Model.maxViscosity = 1e23 * u.pascal * u.second
Model.minViscosity = 1e19 * u.pascal * u.second
 
stickyAirLayer.viscosity = GEO.ConstantViscosity(1e19 * u.pascal * u.second)
for plasticLayer in plasticLayers:
    plasticLayer.viscosity = 1e23 * u.pascal * u.second
    plasticLayer.maxViscosity = 1e23 * u.pascal * u.second
    plasticLayer.minViscosity = 5e19 * u.pascal * u.second

frictionalBasalLayer.viscosity = 1e23 * u.pascal * u.second
rigidBase.viscosity = GEO.ConstantViscosity(1e23 * u.pascal * u.second)
sediment.viscosity = GEO.ConstantViscosity(1e23 * u.pascal * u.second)


for plasticLayer in plasticLayers:
    plasticLayer.plasticity = GEO.DruckerPrager(
        cohesion=20. * u.megapascal,
        cohesionAfterSoftening=4. * u.megapascal,
        frictionCoefficient=np.tan(np.radians(25.0)),
        frictionAfterSoftening=np.tan(np.radians(20.0)),
        epsilon1=0.01,
        epsilon2=0.06
    )

frictionalBasalLayer.plasticity = GEO.DruckerPrager(
    cohesion=0.1 * u.megapascal,
    #cohesionAfterSoftening=0.001 * u.pascal,
    frictionCoefficient=np.tan(np.radians(12.0)),
    frictionAfterSoftening=np.tan(np.radians(6.0)),
    epsilon1=0.01,
    epsilon2=0.06
)
# sediment.plasticity = GEO.DruckerPrager(
#    cohesion=20. * u.megapascal,
#     cohesionAfterSoftening=4. * u.megapascal,
#    frictionCoefficient=np.tan(np.radians(25.0)),
#     frictionAfterSoftening=np.tan(np.radians(15.0)),
#     epsilon1=0.01,
#     epsilon2=0.6
# )

### Velocity Boundary Conditions
The rigid base has the same velocity(=1cm/year,or velocity of the left wall), the linear gradational velocity is in the low frictional layer, this prevents the rigid bottom sheet from bending at the left edge, moreover as the rigid bottom sheet is analogous to a Mylar(PET) conveyor belt / sheet in sandbox models(Konstantinovskaya and Malavieille, 2011[https://doi.org/10.1016/j.tecto.2011.01.020], or Bose et al 2014[https://doi.org/10.1016/j.jsg.2014.07.004]) 

In [ ]:
tapeL=frictionalBasalLayer
flthick=GEO.nd(tapeL.top-tapeL.bottom)
conditions = [(Model.y <= GEO.nd(rigidBase.top), GEO.nd(-velocity)),
              (Model.y < GEO.nd(tapeL.top),
               GEO.nd(-velocity)*(flthick-(Model.y-GEO.nd(tapeL.bottom)))/flthick),
              (True, GEO.nd(0. * u.centimeter / u.year))]

Model.set_velocityBCs(left=[conditions, 0.],
                      right=[-velocity, None],
                      top=[None, None],
                      bottom=[-velocity, 0.])


In [ ]:
figVelocity=glucifer.Figure(store, figsize=figSize, name="Velocity Map",boundingBox=bBox)
figVelocity.Surface(Model.mesh,GEO.uw.function.math.dot(Model.velocityField, Model.velocityField),colours='spectral')

In [ ]:
figSr = glucifer.Figure(
    figsize=figSize,
    quality=figQuality,
    title="Strain Rate (1/s)",
    boundingBox=bBox)
figSr.Points(
    Model.swarm,
    Model._strainRate_2ndInvariant * GEO.Dimensionalize(
        1.0, 1. / u.seconds).magnitude,
    logScale=True,
    colours=lavavu.matplotlib_colourmap("viridis"),
    valueRange=[1e-18, 1e-13],
    fn_mask=Model.materialField > stickyAirLayer.index,
    fn_size=fnSize)
figSr.show()

In [ ]:
figPlasticStrain = glucifer.Figure(store, figsize=figSize, boundingBox=bBox, quality=figQuality,
                               name="Plastic Strain Map", title="Accumulated Plastic Strain Map")
figPlasticStrain.Points(Model.swarm, Model.plasticStrain, 
                        fn_size=fnSize,
                        colours=lavavu.matplotlib_colourmap("magma_r"),
                        fn_mask=Model.materialField > stickyAirLayer.index,
                        valueRange=[0.0, 4.0])
figPlasticStrain.show() 

In [ ]:
if(GEO.uw.nProcs() == 1):
#     GEO.rcParams["solver"] = "superlu"
  GEO.rcParams["solver"] = "lu"
else:
    GEO.rcParams["solver"] = "mumps"

GEO.rcParams["penalty"] = 1e6
GEO.rcParams["nonlinear.tolerance"] = 1e-3
GEO.rcParams["nonlinear.max.iterations"]=200
GEO.rcParams["nonlinear.min.iterations"]=3
GEO.rcParams["initial.nonlinear.tolerance"]=1e-5
GEO.rcParams["initial.nonlinear.max.iterations"]=500
GEO.rcParams["popcontrol.particles.per.cell.2D"]=25

Model.init_model()

#### Basic Hillslope Diffusion, the code is a WIP (expect to have a similar API as below)

In [ ]:
#Model.surfaceProcesses = GEO.surfaceProcesses.BasicHillSlopeDiffsuion2d(
#    Model=Model,
#    airIndex=stickyAirLayer.index,
#    sedimentIndex=sediment.index,
#    diffusivity=GEO.nd(1e-8 * u.metre**2 / u.second))

In [ ]:
Model.run_for(2.0*u.megayears,restartStep=None)

In [ ]:
figSr.show()

In [ ]:
figVelocity.show()

In [ ]:
figViscosity = glucifer.Figure(store, figsize=figSize, boundingBox=bBox, quality=figQuality,
                               name="Viscosity Map", title="Viscosity Map (Pa.s)")
figViscosity.Points(Model.swarm,GEO.Dimensionalize(Model.viscosityField, u.pascal * u.second),
                    logScale=True,
                    colours=lavavu.matplotlib_colourmap("YlGnBu_r"),
                    fn_size=fnSize,valueRange=[1e19, 1e23],
                    fn_mask=Model.materialField > stickyAirLayer.index)
figViscosity.objects[0].colourBar["tickvalues"] = [1e19, 1e20, 1e21, 1e22, 1e23]

figViscosity.show()

In [ ]:
figPlasticStrain.show()

In [ ]:
figMaterial.show()